In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

# 预训练的开源大模型
# model = AutoModelForCausalLM.from_pretrained(
#     "/mnt/data/xuhu/qwen/Qwen2-7B-Instruct",
#     torch_dtype="auto",
#     device_map="auto"
# )
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("/mnt/data/xuhu/qwen/Qwen2-7B-Instruct")

/home/xuhu/miniconda3/envs/qwen2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.70s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import pandas as pd
dc = pd.read_csv('../dataset/en-zh.dic', sep='\t', header=None).set_index(0).to_dict()[1]

from tqdm import tqdm
lines = open('../dataset/test_en.txt').readlines()

In [ ]:
result = []
for line in tqdm(lines):
    sp_words = [x for x in line.lower().split() if x in dc.keys()]
    sp_words_meaning = [dc[x] for x in sp_words]

    sp_prompt = '文章字符为：'
    if len(sp_words) > 0:
        for x, y in zip(sp_words, sp_words_meaning): 
            sp_prompt += f'{x} 翻译为 {y}; '
    # 主要任务
    messages = [
        {"role": "system", "content": "将英文翻译为中文，不要有其他输出，直接输出翻译后的文本。保留特殊单词的翻译。"},
    ]

    # 人工的词典的规则
    if len(sp_prompt) > 0:
        messages.append({"role": "user", "content": sp_prompt})
    messages.append({"role": "user", "content": f"待翻译文本（从英文翻译为中文）：{line}"})
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    result_line = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    result.append(result_line)
    
with open('../results/qwen_submit.txt', 'w') as f:
    for line in result:
        line = line.strip().replace('\n', '')
        print(line)
        f.write(line + '\n')